# Assignment 2A, Part 2: Retrieval

Implement BM25 and LM retrieval.

In [1]:
QUERY_FILE = "data/queries.txt"  # make sure the query file exists on this location
OUTPUT_FILE = "data/output.csv"  # output the ranking

## Load index

In [4]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk, unicodedata, re, csv, hashedindex, string, os
import re
import gzip
from bs4 import BeautifulSoup
import numpy as np
import glob
import pandas as pd
import math
from IPython.display import clear_output 
import pickle

In [5]:
inv_idx = pickle.load(open("data/backup/indexer.p", "rb" ))

total_doc_length = 0
for doc in inv_idx.documents():
    total_doc_length += inv_idx.get_document_length(doc)
    
average_doc_length =  total_doc_length / len(inv_idx.documents())
NUM_DOCS = len(inv_idx.documents())
collection_document_length = 0 #total length of documents in the collection


### Load the queries from the file

See the assignment description for the format of the query file [here](https://github.com/kbalog/uis-dat640-fall2019/tree/master/assignments/assignment-2a).

In [6]:
def load_queries(query_file):
    queries = {}
    with open(query_file, "r") as fin:
        for line in fin.readlines():
            qid, query = line.strip().split(" ", 1)
            queries[qid] = query
    return queries

In [7]:
queries = load_queries(QUERY_FILE)

## Retrieval models

In [8]:
# nt - number of documents that contain t
def bm25_idf(doc_containing_query_term):
    val = 1 + ((NUM_DOCS - doc_containing_query_term + 0.5)/(doc_containing_query_term + 0.5))
    return math.log(val)

def tokenize_query_text(query):
    clean_tokens = []
    query = unicodedata.normalize('NFKD', query).encode('ascii', 'ignore').decode('utf-8', 'ignore') # remove non ascii
    query = query.lower() #convert to lowercase
    query = query.translate(str.maketrans('', '', string.punctuation)) #remove punctuations
    for token in nltk.word_tokenize(query):
        # skip stop words
        if token in set(stopwords.words('english')):
            continue

        token = PorterStemmer().stem(token) # stem    
        clean_tokens.append(token)

    return clean_tokens

In [9]:
def bm25(query, k = 1.2, b = 0.75, number_of_results=100, get_score = True):
    bm25_query_doc_scores = {} #stores score for each document for the provided query
    term_score_summation = 0

    for query_term in tokenize_query_text(query):
        #check if we have indexed this query term
        if query_term in inv_idx.terms():
            doc_containing_query_term = len(inv_idx.get_documents(query_term))
            for document in inv_idx.get_documents(query_term):
                ft_d = inv_idx.get_term_frequency(query_term,document)
                doc_len = inv_idx.get_document_length(document)

                doc_score_numerator = (ft_d * (1 + k))
                doc_score_denomenator = (ft_d + k * (1 - b + (b * (doc_len/average_doc_length))))
                doc_score_left = (doc_score_numerator/doc_score_denomenator)

                score = (doc_score_left* bm25_idf(doc_containing_query_term))

                if(document in bm25_query_doc_scores):
                    bm25_query_doc_scores[document] += score
                else:
                    bm25_query_doc_scores[document] = score
    
    sorted_list = sorted(bm25_query_doc_scores.items(), key=lambda score: score[1], reverse = True)[:number_of_results]
    if not get_score:
        relevant_articles = []
        for x in sorted_list:
            relevant_articles.append(x[0])

        return relevant_articles
    return sorted_list


In [10]:
def create_collection_lm():
    CLM = {}
    for term in inv_idx.terms():
        total_term_frequency = inv_idx.get_total_term_frequency(term)
        
        CLM[term] = total_term_frequency / total_doc_length
    
    return CLM

In [11]:
CLM = create_collection_lm()

In [12]:
def LM_jelinek_merker_smoothing(query,Lamda=0.5, number_of_results=100, get_score=True):
    lm_query_doc_scores = {} #stores score for each document for the provided query
    query_list=tokenize_query_text(query)
    for query_term in query_list:
        f_tq=query_list.count(query_term)
        
        if query_term in inv_idx.terms():
            for (doc_id, f_td) in inv_idx[query_term].items():
                doc_length=inv_idx.get_document_length(doc_id)
                term_score = round(math.log((1-Lamda)*(f_td / doc_length) + (Lamda*CLM[query_term])),3)
                lm_query_doc_scores[doc_id] = lm_query_doc_scores.get(doc_id, 0) + term_score
            lm_query_doc_scores[doc_id] = f_tq * lm_query_doc_scores[doc_id]
    
    sorted_list = sorted(lm_query_doc_scores.items(), key=lambda score: score[1], reverse = True)[:number_of_results]
    if not get_score:
        relevant_articles = []
        for x in sorted_list:
            relevant_articles.append(x[0])

        return relevant_articles
    return sorted_list

In [13]:
def LM_dirichilet(query, number_of_results=100, get_score=True, meu = average_doc_length):
    doc_scores = {} #stores score for each document for the provided query
    query_list=tokenize_query_text(query)
    for query_term in query_list:
        f_tq=query_list.count(query_term)
        
        if query_term in inv_idx.terms():
            for (doc_id, f_td) in inv_idx[query_term].items():
                doc_length=inv_idx.get_document_length(doc_id)
                
                term_score = round(math.log((f_td+meu*CLM[query_term])/(doc_length + meu)),3)
                doc_scores[doc_id] = doc_scores.get(doc_id, 0) + term_score
            doc_scores[doc_id] = f_tq * doc_scores[doc_id]

    sorted_list = sorted(doc_scores.items(), key=lambda score: score[1], reverse = True)[:number_of_results]
    if not get_score:
        relevant_articles = []
        for x in sorted_list:
            relevant_articles.append(x[0])

        return relevant_articles
    return sorted_list

### Perform retrieval

**TODO** Generate a ranking for each query and output the results to `OUTPUT_FILE`

See the assignment description for the format of the output file [here](https://github.com/kbalog/uis-dat640-fall2019/tree/master/assignments/assignment-2a).

In [14]:
def save_output(results,filename=OUTPUT_FILE):
    with open(filename, mode='w') as index_file:
        csv_writer = csv.writer(index_file, delimiter=',', quotechar='"')
        csv_writer.writerow(["QueryId","DocumentId"])
        for term in results:
            for article in results[term]:
                csv_writer.writerow([term,article]) #save as counter object

In [15]:
# Compute BM25 and save to file
results_bm25={}

for q_id, query in queries.items():
    results_bm25[q_id] = bm25(query, get_score=False, k=1.2, b=.3)
    clear_output()
print("completed ranking")

save_output(results_bm25, "data/bm25_output.csv")

completed ranking


In [19]:
# results_LMJM
results_LMJM={}

for q_id, query in queries.items():
    results_LMJM[q_id] = LM_jelinek_merker_smoothing(query,Lamda=0.1, number_of_results=100, get_score=False)
print("completed ranking")

save_output(results_LMJM, "data/lm_jm_output.csv")

completed ranking


In [22]:
results_LMD={}

for q_id, query in queries.items():
    results_LMD[q_id] = LM_dirichilet(query, number_of_results=100, get_score=False, meu = 10)
print("completed ranking")

save_output(results_LMD, "data/lm_dir_output.csv")

completed ranking


## Evaluation

Report on the evaluation results (using the [Evaluation notebook](1_Evaluation.ipynb)) here.

Describe the parameter settings used for the two methods: *TODO*

Write the name of the corresponding output file in the table. These files should be pushed to your repository.


| **Method** | **Output file** | **P@10** | **MAP** | **MRR** |
| -- | -- | -- | -- | -- | -- |
| BM25 | `data/bm25_output.csv` | *0.220* | *0.082* | *0.356* |
| LM | `data/lm_dir_output.csv` | *0.016* | *0.002* | *0.040* |

    
